In [1]:
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm
from glob import glob
import ast, json
from bs4 import BeautifulSoup
#specify home folder (all sub-folders, scripts, data in this folder)
data_folder = ''
api = ''
date = ''

### extract potential ades

In [94]:
serious_ade_df = pd.read_csv(data_folder+'data/rx_drug_serious_ade_raw.csv')
serious_ade_df = serious_ade_df[serious_ade_df.text.notna()]
serious_ade_df['text'] = serious_ade_df['text'].apply(lambda x: BeautifulSoup(x, 'html.parser'))
serious_ade_df['subtitle'] = serious_ade_df.text.apply(lambda x: x.find(class_='subtitle').text
                                                       if len(x.find_all(class_='subtitle'))>0 else None)
serious_ade_df['subtitle'] = serious_ade_df.apply(lambda x: x.text.find('h5').text
                                                  if (x.subtitle is None and len(x.text.find_all('h5'))) > 0
                                                                   else x.subtitle, axis=1)
serious_ade_df['content'] = serious_ade_df.text.apply(lambda x: x.find(class_='block1').text
                                                      if len(x.find_all(class_='block1'))>0 else None)
serious_ade_df['content'] = serious_ade_df.apply(lambda x: x.text.text.replace(x.subtitle, '').strip()
                                                  if x.content is None and x.subtitle != '重大な副作用'
                                                     and x.subtitle is not None
                                                                   else x.content, axis=1)
print(serious_ade_df.shape)
serious_ade_df.head()

(16800, 4)


,japic_code,text,subtitle,content
0,55998,"[[, [重大な副作用], , , [\n, [ショック、アナフィラキシー（呼吸困難、不快感...",重大な副作用,\nショック、アナフィラキシー（呼吸困難、不快感、浮腫、潮紅、蕁麻疹等）（0.1％未満）があ...
1,55997,"[[, [11.1 重大な副作用], , , [[次の副作用があらわれることがあるので、観察...",11.1 重大な副作用,"[, 次の副作用があらわれることがあるので、観察を十分に行い、異常が認められた場合には投与を..."
2,54447,"[[, [重大な副作用], , , [\n, [次のような症状があらわれることがあるので、投...",重大な副作用,\n次のような症状があらわれることがあるので、投与初期は1〜2週間ごとを目安に、その後も頻回...
3,54413,"[[, [重大な副作用], , , [\n, [（頻度不明）], \n, [ショック、アナフ...",重大な副作用,\n（頻度不明）\nショック、アナフィラキシー\n\nショック、アナフィラキシー（呼吸困難、...
4,54424,"[[, [重大な副作用], , , [\n, [骨髄抑制], \n, [\n, <p>汎血球...",重大な副作用,\n骨髄抑制\n\n汎血球減少、白血球減少、血小板減少、貧血があらわれることがあるので、投与...


In [69]:
#because we have strings of data table cells, we just look into the unique cell strings. (there is a lot of overlap)
unique_ades = serious_ade_df[serious_ade_df.content.notna()][['content']].drop_duplicates()
print(unique_ades.shape)
unique_ades.head(1)

(4924, 1)


,content
0,\nショック、アナフィラキシー（呼吸困難、不快感、浮腫、潮紅、蕁麻疹等）（0.1％未満）があ...


In [70]:
#prep for mapping
external_data_folder = './external_data/'
#read in the meddra map file
umls_map = pd.read_csv(external_data_folder+'umls_meddra_jp.csv')
umls_mjp_str_sdui = dict(zip(umls_map.STR, umls_map.SDUI)) #dict of meddra str -> sdui
#read in the mesh-jp map file
umls_mshjp = pd.read_csv(external_data_folder+'umls_mshjpn.csv')
umls_mshjp = umls_mshjp[['CUI', 'STR']].merge(umls_map[['CUI', 'SDUI']], on = 'CUI', how = 'inner')
umls_mshjp_str_cui = dict(zip(umls_mshjp.STR, umls_mshjp.SDUI)) #dict of mesh-jp -> meddra sdui
umls_map.head(1)

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,C0000727,JPN,P,L3471380,PF,S3998874,N,A11418933,NaN,NaN,10000647,MDRJPN,PT,10000647,急性腹症,3,N,NaN


In [ ]:
meddra_found_ades = []
mesh_found_ades = []
meddra_names = umls_map.STR.unique().tolist()
mesh_names = umls_mshjp.STR.unique().tolist()

for ade_text in tqdm(unique_ades.content.tolist()):
  ar_text = ade_text
  meddra_found_terms = []
  mesh_found_terms = []
  #iterate through list of meddra concepts, if found - add to list
  for concept_name in meddra_names:
    if ar_text.find(concept_name) == -1:
      continue
    else:
      meddra_found_terms.append(concept_name)
  #check to see if term is in MeSH too (extra coverage)
  for concept_name in mesh_names:
    if ar_text.find(concept_name) == -1:
      continue
    else:
      mesh_found_terms.append(concept_name)
  mesh_found_ades.append(mesh_found_terms)
  meddra_found_ades.append(meddra_found_terms)

In [73]:
unique_ades['meddra_ade_list'] = meddra_found_ades
unique_ades['mesh_ade_list'] = mesh_found_ades
#unique_ades['ade_combination_num'] = unique_ades.ade_list.apply(lambda x: len(x) if x!= None else None)
unique_ades.to_csv(data_folder+'data/ade/rx_raw_serious_ade_extraction.csv', index=False)
unique_ades.head()

,content,meddra_ade_list,mesh_ade_list
0,\nショック、アナフィラキシー（呼吸困難、不快感、浮腫、潮紅、蕁麻疹等）（0.1％未満）があ...,"[アナフィラキシー, 呼吸困難, 浮腫, 潮紅, 麻疹, ショック, 蕁麻疹, 不快感]","[アナフィラキシー, 呼吸困難, 浮腫, 潮紅, 麻疹, ショック, 蕁麻疹]"
1,"[, 次の副作用があらわれることがあるので、観察を十分に行い、異常が認められた場合には投与を...","[アナフィラキシー, 血圧, BUN, 認知障害, 咳嗽, サイトメガロウイルス, 下痢, ...","[アナフィラキシー, 血圧, 倦怠, 脳症, 認知障害, 咳嗽, 咳, サイトメガロウイルス..."
2,\n次のような症状があらわれることがあるので、投与初期は1〜2週間ごとを目安に、その後も頻回...,"[無顆粒球症, 貧血, 再生不良性貧血, 巨赤芽球性貧血, 血圧, 悪性新生物, 癌, よう...","[無顆粒球症, 貧血, 再生不良性貧血, 巨赤芽球性貧血, 赤芽球性貧血, 血圧, 脳症, ..."
3,\n（頻度不明）\nショック、アナフィラキシー\n\nショック、アナフィラキシー（呼吸困難、...,"[腹痛, 無顆粒球症, アナフィラキシー, 貧血, 溶血性貧血, 強直, よう, 間代, 大...","[腹痛, 無顆粒球症, アナフィラキシー, 貧血, 溶血性貧血, よう, 大腸炎, 咳嗽, ..."
4,\n骨髄抑制\n\n汎血球減少、白血球減少、血小板減少、貧血があらわれることがあるので、投与...,"[アナフィラキシー, 貧血, 溶血性貧血, よう, 線維症, 心停止, 血液検査, 溶血, ...","[アナフィラキシー, 貧血, 溶血性貧血, よう, 線維症, 心停止, 血液検査, 溶血, ..."


----
### translation

In [ ]:
#run gpt using local environment

### extract ade terms

In [83]:
ade_df = pd.read_csv(data_folder+'data/ade/rx_raw_serious_ade_translated.csv')
ade_df['gpt_output'] = ade_df.gpt_output.apply(lambda x: x.lower())
ade_df.head(1)

,content,gpt_output
0,\nショック、アナフィラキシー（呼吸困難、不快感、浮腫、潮紅、蕁麻疹等）（0.1％未満）があ...,"shock, anaphylaxis (difficulty breathing, disc..."


In [84]:
f = '/'
meddra_df = pd.read_csv(f+'external_data/umls_meddra_en.csv')
meddra_df['STR'] = meddra_df.STR.apply(lambda x: x.lower())
meddra_df['len'] = meddra_df.STR.apply(lambda x: len(x))
print(meddra_df.shape[0])
meddra_df = meddra_df[(meddra_df['len'] >= 5)]
meddra_df = meddra_df[(meddra_df.TTY.isin(['PT', 'LLT']))]
print(meddra_df.shape[0])
meddra_dict = dict(zip(meddra_df.STR, meddra_df.SDUI))
meddra_df.head(1)

116807
100897


,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF,len
0,C0000727,ENG,P,L0000727,VCW,S0584932,N,A0639292,NaN,NaN,10000647,MDR,PT,10000647,acute abdomen,3,N,256.0,13


In [85]:
app_data = []
meddra_names = meddra_df.STR.unique().tolist()
for ade_text in tqdm(ade_df.gpt_output.tolist()):
  if str(ade_text) != 'nan':
    ade_text = ade_text.lower()
    meddra_found_terms = []
    #iterate through list of meddra concepts, if found - add to list
    for concept_name in meddra_names:
      if ade_text.find(concept_name) == -1:
        continue
      else:
        i = ade_text.index(concept_name)
        meddra_found_terms.append([i, concept_name, meddra_dict[concept_name]])
    app_data.append([ade_text, meddra_found_terms])

100%|██████████| 4055/4055 [07:32<00:00,  8.97it/s]


In [87]:
app_data_df = pd.DataFrame(app_data, columns = ['string', 'list'])
app_data_df = app_data_df.explode('list')
app_data_df['location'] = app_data_df['list'].apply(lambda x: x[0] if str(x) != 'nan' else None)
app_data_df['found_term'] = app_data_df['list'].apply(lambda x: x[1] if str(x) != 'nan' else None)
app_data_df['meddra_id'] = app_data_df['list'].apply(lambda x: x[2] if str(x) != 'nan' else None)
app_data_df = app_data_df.drop(['list'], axis = 1)
app_data_df.to_csv(data_folder+'data/ade/rx_raw_serious_ade_matched_terms.csv', index=False)
app_data_df.head(1)

,string,location,found_term,meddra_id
0,"shock, anaphylaxis (difficulty breathing, disc...",7.0,anaphylaxis,10002198.0


In [88]:
building_strings = []
app_data_df = app_data_df.rename(columns = {'string':'ar_text'})
for i, row in tqdm(app_data_df.iterrows()):
  if str(row['location']) != 'nan':
    term, start_pos, ar_text = row['found_term'], int(row['location']), row['ar_text']
    #default settings
    nwords, prop_before = 125, 0.125
    #pull the full text
    #ar_text = ade_text_table_dict[label_id]

    term_nwords = len(term)
    size_before = max(int((nwords-2*term_nwords)*prop_before), 1)
    size_after = max(int((nwords-2*term_nwords)*(1-prop_before)), 1)

    before_text = ar_text[:start_pos]
    after_text = ar_text[(start_pos+term_nwords):]

    before_parts = before_text.split()[-1*size_before:]
    after_parts = after_text.split()[:size_after]

    li = [term]
    li.extend(before_parts)
    li.append('EVENT')
    li.extend(after_parts)
    example_string = ' '.join(li)
    building_strings.append(example_string)
  else:
    building_strings.append(None)
app_data_df['string'] = building_strings
app_data_df.to_csv(data_folder+'data/ade/rx_raw_serious_ade_matched_terms.csv', index=False)

98312it [00:11, 8224.53it/s] 


In [95]:
serious_ade_df = pd.read_csv(data_folder+'data/rx_drug_serious_ade_raw.csv')
serious_ade_df = serious_ade_df[serious_ade_df.text.notna()]
serious_ade_df['text'] = serious_ade_df['text'].apply(lambda x: BeautifulSoup(x, 'html.parser'))
serious_ade_df['subtitle'] = serious_ade_df.text.apply(lambda x: x.find(class_='subtitle').text
                                                       if len(x.find_all(class_='subtitle'))>0 else None)
serious_ade_df['subtitle'] = serious_ade_df.apply(lambda x: x.text.find('h5').text
                                                  if (x.subtitle is None and len(x.text.find_all('h5'))) > 0
                                                                   else x.subtitle, axis=1)
serious_ade_df['content'] = serious_ade_df.text.apply(lambda x: x.find(class_='block1').text
                                                      if len(x.find_all(class_='block1'))>0 else None)
serious_ade_df['content'] = serious_ade_df.apply(lambda x: x.text.text.replace(x.subtitle, '').strip()
                                                  if x.content is None and x.subtitle != '重大な副作用'
                                                     and x.subtitle is not None
                                                                   else x.content, axis=1)
print(serious_ade_df.shape)
serious_ade_df.head(1)

(16800, 4)


,japic_code,text,subtitle,content
0,55998,"[[, [重大な副作用], , , [\n, [ショック、アナフィラキシー（呼吸困難、不快感...",重大な副作用,\nショック、アナフィラキシー（呼吸困難、不快感、浮腫、潮紅、蕁麻疹等）（0.1％未満）があ...


In [99]:
ade_df = pd.read_csv(data_folder+'data/ade/rx_raw_serious_ade_translated.csv')
ade_df['ar_text'] = ade_df['gpt_output'].apply(lambda x: x.lower())
app_data_df = pd.read_csv(data_folder+'data/ade/rx_raw_serious_ade_matched_terms.csv')
exact_terms_df = ade_df.merge(app_data_df, how='left', on='ar_text')
exact_terms_df = serious_ade_df[['japic_code', 'content', 'subtitle']].merge(exact_terms_df, on = 'content', how = 'left').rename(columns={'japic_code':'drug'})
exact_terms_df['section'] = 'AR'
exact_terms_df['set_id'] = exact_terms_df['drug']
folder = './'
llt_pt = pd.read_csv(folder+'meddra_llt_pt_map.txt', delimiter = '|')
llt_pt_id_dict = dict(zip(llt_pt.llt_concept_code, llt_pt.pt_concept_code))
llt_pt_term_dict = dict(zip(llt_pt.llt_concept_code, llt_pt.pt_concept_name))
exact_terms_df['pt_meddra_id'] = exact_terms_df.meddra_id.apply(lambda x: llt_pt_id_dict[x] if x in llt_pt_id_dict.keys() else None)
exact_terms_df['pt_meddra_term'] =  exact_terms_df.meddra_id.apply(lambda x: llt_pt_term_dict[x] if x in llt_pt_term_dict.keys() else None)
print(exact_terms_df.shape)
exact_terms_df = exact_terms_df.drop(columns = ['gpt_output', 'ar_text'])
exact_terms_df.to_csv(data_folder+'data/serious_ade/sentences-rx_method14_nwords125_clinical_bert_application_set_AR_v0120.csv', index=False)
exact_terms_df.head()

(348323, 13)


,drug,content,subtitle,location,found_term,meddra_id,string,section,set_id,pt_meddra_id,pt_meddra_term
0,55998,\nショック、アナフィラキシー（呼吸困難、不快感、浮腫、潮紅、蕁麻疹等）（0.1％未満）があ...,重大な副作用,7.0,anaphylaxis,10002198.0,"anaphylaxis shock, EVENT (difficulty breathing...",AR,55998,10002198.0,Anaphylactic reaction
1,55998,\nショック、アナフィラキシー（呼吸困難、不快感、浮腫、潮紅、蕁麻疹等）（0.1％未満）があ...,重大な副作用,20.0,difficulty breathing,10013968.0,"difficulty breathing shock, anaphylaxis ( EVEN...",AR,55998,10013968.0,Dyspnoea
2,55998,\nショック、アナフィラキシー（呼吸困難、不快感、浮腫、潮紅、蕁麻疹等）（0.1％未満）があ...,重大な副作用,54.0,edema,10030095.0,"edema shock, anaphylaxis (difficulty breathing...",AR,55998,10030095.0,Oedema
3,55998,\nショック、アナフィラキシー（呼吸困難、不快感、浮腫、潮紅、蕁麻疹等）（0.1％未満）があ...,重大な副作用,61.0,flushed,10016825.0,"flushed shock, anaphylaxis (difficulty breathi...",AR,55998,10016825.0,Flushing
4,55998,\nショック、アナフィラキシー（呼吸困難、不快感、浮腫、潮紅、蕁麻疹等）（0.1％未満）があ...,重大な副作用,61.0,flushed skin,10016825.0,"flushed skin shock, anaphylaxis (difficulty br...",AR,55998,10016825.0,Flushing


In [100]:
exact_terms_df = pd.read_csv(data_folder+'data/serious_ade/sentences-rx_method14_nwords125_clinical_bert_application_set_AR_v0120.csv')
exact_terms_df = exact_terms_df[exact_terms_df.string.notna()]
exact_terms_df.to_csv(data_folder+'data/serious_ade/sentences-rx_method14_nwords125_clinical_bert_application_set_AR_v0120.csv', index=False)

### run onsides

In [2]:
#prep onsides materials
folder = '/onsides'
ar_model = folder + '/bestepoch-bydrug-PMB_14-AR-125-all_222_24_25_2.5e-05_256_32.pth'
bw_model = folder + '/bestepoch-bydrug-PMB_14-AR-125-all_222_24_25_2.5e-05_256_32.pth'
import os
os.chdir('/onsides-2.0.0/')
!python3 -m pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.3/292.3 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 121.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 27.8 M

In [3]:
#use onsides model
f = data_folder+'data/serious_ade/sentences-rx_method14_nwords125_clinical_bert_application_set_AR_v0120.csv'
!python3 src/predict.py --model $ar_model --examples $f

2024-01-27 02:41:39.503754: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-27 02:41:39.557501: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-27 02:41:39.557545: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-27 02:41:39.559457: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-27 02:41:39.567741: I tensorflow/core/platform/cpu_feature_guar